# This notebook is to prepare streets.csv dataset that will store the names of the Yerevan streets and their coordinates for further use.

In [24]:
import osmnx as ox
import pandas as pd
from unidecode import unidecode
from geopy.distance import geodesic
from shapely.geometry import LineString
from fuzzywuzzy import fuzz

In [47]:
def get_line_midpoint(coords):
    """
    Get the midpoint coordinates (x, y) of a LINESTRING defined by a list of coordinates.
    
    Args:
        coords (list of tuples): List of (x, y) coordinates representing the LINESTRING.
        
    Returns:
        tuple: Tuple containing the x and y coordinates of the midpoint.
    """
    line = LineString(coords)
    midpoint = line.interpolate(0.5, normalized=True)
    return (midpoint.x, midpoint.y)

def encode_names(x):
    unicoded = unidecode(str(x))
    unicoded = unicoded.replace("poghots", "street")
    return unicoded

In [28]:
place_name = "Yerevan, Armenia"
graph = ox.graph_from_place(
    place_name,
    network_type='{"walk"}',
    custom_filter='["name:en"]'
)

In [60]:
gdfs = ox.utils_graph.graph_to_gdfs(graph, edges = True)
streets = gdfs[1]
streets["name_am"] = streets["name"]
streets["name_en"] = streets["name"].apply(encode_names)
streets["name_en"] = streets["name_en"].str.replace("`", "")
streets["coordinates"] = streets["geometry"].apply(get_line_midpoint)
streets["coordinates"] = streets["coordinates"].apply(lambda coords: (coords[1], coords[0]))
streets = streets.reset_index()
streets = streets[["name_am", "name_en", "coordinates"]]
streets = streets.dropna()

In [61]:
streets.to_csv('streets.csv', index=False)